# 모델(roberta-base)의 custom 및 f1_score & pearsonr 

Team LostCow의 roberta모델을 기준으로 함

출처: https://github.com/l-yohai/KLUE

In [1]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.8 MB 10.3 MB/s 
     |████████████████████████████████| 895 kB 48.8 MB/s 
     |████████████████████████████████| 67 kB 6.0 MB/s 
     |████████████████████████████████| 6.5 MB 42.3 MB/s 
     |████████████████████████████████| 596 kB 57.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 8.9 MB/s 
     |████████████████████████████████| 212 kB 66.2 MB/s 
     |████████████████████████████████| 134 kB 73.2 MB/s 
     |████████████████████████████████| 1.1 MB 60.8 MB/s 
     |████████████████████████████████| 127 kB 75.4 MB/s 
     |████████████████████████████████| 144 kB 72.6 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████| 271 kB 73.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urlli

In [2]:
import tarfile

#훈련 데이터셋 다운로드 및 압축풀기

!wget https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000067/data/klue-sts-v1.1.tar.gz

tar_bz2_file = tarfile.open("/content/klue-sts-v1.1.tar.gz")
tar_bz2_file.extractall(path="/content")
tar_bz2_file.close()

--2022-03-17 06:28:46--  https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000067/data/klue-sts-v1.1.tar.gz
Resolving aistages-prod-server-public.s3.amazonaws.com (aistages-prod-server-public.s3.amazonaws.com)... 52.92.165.217
Connecting to aistages-prod-server-public.s3.amazonaws.com (aistages-prod-server-public.s3.amazonaws.com)|52.92.165.217|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1349881 (1.3M) [application/x-gzip]
Saving to: ‘klue-sts-v1.1.tar.gz’

klue-sts-v1.1.tar.g 100%[===================>]   1.29M  --.-KB/s    in 0.07s   

2022-03-17 06:28:46 (17.8 MB/s) - ‘klue-sts-v1.1.tar.gz’ saved [1349881/1349881]



In [3]:
import torch

#GPU 사용 여부 확인 및 name 확인

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


In [4]:
import json

#train 데이터셋을 train,valid 데이터셋으로 split

def read_json(file_path):
    with open(file_path) as f:
        return json.load(f)

data=read_json('/content/klue-sts-v1.1/klue-sts-v1.1_train.json')

train_length=int(len(data)*0.9)
train=data[:train_length]
vaild=data[train_length:]

print('data_set:',len(data))
print('train_set:',len(train),', valid_set:',len(vaild))

with open('train_split.json','w') as f:
  json.dump(train,f,ensure_ascii = False)
with open('valid_split.json','w') as f:
  json.dump(vaild,f,ensure_ascii = False)

data_set: 11668
train_set: 10501 , valid_set: 1167


In [5]:
#모델 custom을 진행한 폴더 압축해제
!unzip /content/KLUE_custom.zip

Archive:  /content/KLUE_custom.zip
  inflating: sts/__pycache__/dataset.cpython-37.pyc  
  inflating: sts/__pycache__/metric.cpython-37.pyc  
  inflating: sts/__pycache__/model.cpython-37.pyc  
  inflating: sts/__pycache__/utils.cpython-37.pyc  
  inflating: sts/data/download.sh    
  inflating: sts/README.md           
  inflating: sts/dataloader.py       
  inflating: sts/dataset.py          
  inflating: sts/infer_test.ipynb    
  inflating: sts/inference.py        
  inflating: sts/metric.py           
  inflating: sts/model.py            
 extracting: sts/requirements.txt    
  inflating: sts/train.py            
  inflating: sts/utils.py            


In [13]:
# custom 모델 학습
!python /content/sts/train.py --data_dir /content --model_name_or_path "klue/roberta-base" --train_filename "train_split.json" --valid_filename "valid_split.json" --num_train_epochs 10 --save_steps 100 --save_steps 100

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForStsRegression: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForStsRegression from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForStsRegression from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForStsRegression were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['sentence_fc_layer3.linear.weight', 'sentence_fc_layer2.linear.bias', 'roberta.pooler.dense.weight'

In [30]:
#학습한 모델을 이용하여 dev_set 예측값 출력(real_label)
!python /content/sts/inference.py --test_filename "/content/klue-sts-v1.1/klue-sts-v1.1_dev.json" --output_dir "/content"  --model_tar_file "klue-sts-v1.1.tar.gz"

In [35]:
from sklearn.metrics import classification_report,f1_score
import pandas as pd

#valid의 label 값을 추출

valid_data=read_json('/content/klue-sts-v1.1/klue-sts-v1.1_dev.json')
valid_label=[data['labels']['binary-label'] for data in valid_data]


#Regression pred 를 binary-label pred로 변환 및 혼돈 메트릭스 및 f1_score 시각화

df_pred = pd.read_csv('/content/output.csv',header=None)

pred=(df_pred>2.5).astype(int)
print("epochs 10 / >2.5")
print(classification_report(valid_label, pred))
print('base_line(roberta-base) f1_score:',f1_score(valid_label, pred),end='\n\n')

pred=(df_pred>2.85).astype(int)
print("epochs 10 / >2.85")
print(classification_report(valid_label, pred))
print('base_line(roberta-base) f1_score:',f1_score(valid_label, pred))

epochs 10 / >2.5
              precision    recall  f1-score   support

           0       0.99      0.71      0.83       299
           1       0.71      0.99      0.83       220

    accuracy                           0.83       519
   macro avg       0.85      0.85      0.83       519
weighted avg       0.87      0.83      0.83       519

base_line(roberta-base) f1_score: 0.8304761904761905

epochs 10 / >2.85
              precision    recall  f1-score   support

           0       0.97      0.80      0.88       299
           1       0.78      0.97      0.86       220

    accuracy                           0.87       519
   macro avg       0.88      0.88      0.87       519
weighted avg       0.89      0.87      0.87       519

base_line(roberta-base) f1_score: 0.8640973630831643


In [36]:
from datasets import load_metric

#dev_set에 대한 pearsonr score 출력 

df_pred = pd.read_csv('/content/output.csv',header=None)
valid_data=read_json('/content/klue-sts-v1.1/klue-sts-v1.1_dev.json')
valid_label=[data['labels']['real-label'] for data in valid_data]

pearson = load_metric("pearsonr").compute
metric = pearson(predictions=df_pred.to_numpy(), references=valid_label)

print(metric)

{'pearsonr': 0.8981787692339827}
